In [4]:
!pip install pandas_datareader yfinance

  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=6cd622170cb9e332bca34736cf9244e1ad4a0b574b4e1e4ab59cb476b11b5b01
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8375 sha256=2e3f415bf772742c02c2604745f013cadc641ad48a715654fb15665176703f54
  Stored in directory: /root/.cache/pip/wheels/ae/25/47/4d68431a7ec1b6c4b5233365934b74c1d4e665bf5f968d363a
Successfully built yfinance multitasking


In [5]:
import pandas as pd
import requests
from pandas_datareader import data as pdr
import yfinance as yf

In [6]:
source_file = "data.xlsx" # исходник данных для получения universe
start = '2017-01-01' # дата начало получения для finance.yahoo.com
end = '2021-09-04' # дата окончания получения для finance.yahoo.com


In [7]:
main_data = pd.read_excel(source_file, index_col=None, engine="openpyxl", parse_dates=True, sheet_name = "universe")

naufor_df = main_data[main_data['source']=="naufor.ru"]
naufor_codes = naufor_df['code'].to_numpy().tolist()

yfinance_df = main_data[main_data['source']=="finance.yahoo.com"]
yfinance_codes = yfinance_df['code'].to_numpy().tolist()


# Функции экстракции

In [8]:
def get_pif_naufor_quotes(code):
    '''
    получаем данные с naufor.ru
    '''
    referer = 'http://pif.naufor.ru/pif.asp?act=view&id={}'.format(code)
    url = 'http://pif.naufor.ru/pif.asp?act=getpayschajson&id={}&begindate=20030313&enddate=20210601'.format(code)
    resp = requests.get(url, headers={'referer': referer})
    
    blank_array = []
    result_dict = resp.json()
    
    for index, date in enumerate(result_dict['Label']): 
        dd = date.split('.')
        blank_array.append({
            'source': 'naufor.ru',
            'code': code,
            'date': "{year}-{month}-{day}".format(year=dd[2], month=dd[1], day=dd[0]),
            'price': result_dict['UnitValue'][index]
        })

    return blank_array

def get_yahoo_finance_quote(ticker, Start, End):
    '''
    получаем данные с finance.yahoo.com
    '''
    yf.pdr_override() 
    data2 = pdr.get_data_yahoo(ticker, start=Start, end=End)
    output = pd.DataFrame(data=data2['Close'])
    output['code'] = ticker
    clr = output.reset_index().to_dict('records')
    res = []
    for row in clr:
        res.append({
            'source': 'finance.yahoo.com',
            'code': ticker,
            'date': row['Date'].strftime('%Y-%m-%d'),
            'price': row['Close']
        })
    return res

# Извлечение данных с naufor

In [9]:
all_data = []
for code in naufor_codes:
    all_data += get_pif_naufor_quotes(code)
    print("[naufor]Ready code: {}".format(code))

[naufor]Ready code: 299
[naufor]Ready code: 60
[naufor]Ready code: 2746
[naufor]Ready code: 4017
[naufor]Ready code: 2880
[naufor]Ready code: 651
[naufor]Ready code: 3830
[naufor]Ready code: 662
[naufor]Ready code: 525
[naufor]Ready code: 63
[naufor]Ready code: 2322
[naufor]Ready code: 3823
[naufor]Ready code: 4394
[naufor]Ready code: 66
[naufor]Ready code: 606
[naufor]Ready code: 256


In [10]:
for code in yfinance_codes:
    all_data += get_yahoo_finance_quote(code, start, end)
    print("[get_yahoo_finance_quote]Ready code: {}".format(code))

[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: GAZP.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: GMKN.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: LKOH.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: MGNT.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: MTSS.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: NLMK.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: NVTK.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: PLZL.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_

In [11]:
df = pd.DataFrame(all_data)
df.to_excel("tickers_prices.xlsx", index=False)